# PRISM v2.8.0 Dynamical Systems Validation

Validation on four classical dynamical systems:

| System | Type | Key Feature |
|--------|------|-------------|
| **Lorenz** | Strange attractor | Sensitive dependence, butterfly effect |
| **Rössler** | Folding chaos | Simpler attractor with band structure |
| **Double Pendulum** | Hamiltonian chaos | Energy-conserving, regime transitions |
| **Lotka-Volterra** | Limit cycles | Predator-prey oscillations |

## Features Tested
- **PR #2**: Numerical stability (DFA-only Hurst, O(h²) boundaries)
- **PR #3**: Regime-aware normalization (rolling Z-score)

In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from prism.db.parquet_store import get_parquet_path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 4)

## Summary Results

```
================================================================================
PRISM v2.8.0 DYNAMICAL SYSTEMS VALIDATION
================================================================================

System              Vectors    Norm%   Raw |Div|   Norm |Div|   Hurst   Lyapunov
--------------------------------------------------------------------------------
Lorenz             1,022,832   29.6%    3,155,518      20.06   1.0208    0.0871
Rössler            1,085,660   51.7%    3,329,847      10.72   1.0200    0.0783
Double Pendulum      289,139   93.6%          244      13.57   1.0185    0.0874
Lotka-Volterra        85,374   84.9%   34,826,924      14.26   1.0208    0.0862

Key Observations:
- All systems show Hurst ≈ 1.02 (persistent chaos)
- All have positive Lyapunov exponents (chaos confirmed)
- Normalized |divergence| comparable: 10-20 range (vs millions raw)
- Z-scores: mean ≈ 0, std ≈ 1 (normalization working)
```

In [ ]:
# Load all systems
systems = [
    ('lorenz', 'Lorenz'),
    ('rossler', 'Rössler'),
    ('pendulum_regime', 'Double Pendulum'),
    ('lotka_volterra', 'Lotka-Volterra'),
]

data = {}
for domain, name in systems:
    try:
        data[name] = {
            'vec': pl.read_parquet(get_parquet_path('vector', 'signal', domain=domain)),
            'raw': pl.read_parquet(get_parquet_path('vector', 'signal_field', domain=domain)),
            'norm': pl.read_parquet(get_parquet_path('vector', 'signal_field_norm', domain=domain)),
        }
        print(f"Loaded {name}: {len(data[name]['vec']):,} vectors")
    except Exception as e:
        print(f"Failed to load {name}: {e}")

In [ ]:
# Comparison table
print("\n" + "=" * 90)
print(f"{'System':<18} {'Vectors':>12} {'Norm %':>8} {'Raw |Div|':>14} {'Norm |Div|':>12} {'Hurst':>8} {'Lyap':>8}")
print("=" * 90)

for name, d in data.items():
    vec = d['vec']
    raw = d['raw']
    norm = d['norm']
    
    norm_pct = 100 * vec.filter(pl.col('metric_value_norm').is_not_null()).height / len(vec)
    raw_div = raw['divergence'].abs().mean()
    norm_div = norm['divergence'].abs().mean()
    
    hurst = vec.filter((pl.col('engine') == 'hurst') & (pl.col('metric_name') == 'hurst_exponent'))['metric_value'].mean()
    lyap = vec.filter((pl.col('engine') == 'lyapunov') & (pl.col('metric_name') == 'lyapunov_exponent'))['metric_value'].mean() or 0
    
    print(f"{name:<18} {len(vec):>12,} {norm_pct:>7.1f}% {raw_div:>14,.0f} {norm_div:>12.2f} {hurst:>8.4f} {lyap:>8.4f}")

In [ ]:
# Plot divergence distributions for all systems
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (name, d) in enumerate(data.items()):
    # Raw divergence
    div_raw = d['raw'].group_by(['signal_id', 'window_end']).agg(
        pl.col('divergence').first()
    )['divergence'].to_numpy()
    
    axes[0, i].hist(div_raw, bins=50, alpha=0.7, color='steelblue')
    axes[0, i].set_title(f'{name}: Raw')
    axes[0, i].set_xlabel('Divergence')
    
    # Normalized divergence
    div_norm = d['norm'].group_by(['signal_id', 'window_end']).agg(
        pl.col('divergence').first()
    )['divergence'].to_numpy()
    
    axes[1, i].hist(div_norm, bins=50, alpha=0.7, color='darkorange')
    axes[1, i].set_title(f'{name}: Normalized')
    axes[1, i].set_xlabel('Divergence (Z-score)')

axes[0, 0].set_ylabel('Count (Raw)')
axes[1, 0].set_ylabel('Count (Normalized)')
plt.tight_layout()
plt.show()

## System Details

In [ ]:
# Detailed stats per system
for name, d in data.items():
    vec = d['vec']
    print(f"\n{'=' * 60}")
    print(f"{name.upper()}")
    print(f"{'=' * 60}")
    print(f"Signals: {vec['signal_id'].unique().to_list()}")
    print(f"Engines:    {vec['engine'].n_unique()}")
    print(f"Date range: {vec['obs_date'].min()} to {vec['obs_date'].max()}")
    
    norm_valid = vec.filter(pl.col('metric_value_norm').is_not_null())
    print(f"\nNormalization:")
    print(f"  Coverage: {100*len(norm_valid)/len(vec):.1f}%")
    print(f"  Z-score mean: {norm_valid['metric_value_norm'].mean():.4f}")
    print(f"  Z-score std:  {norm_valid['metric_value_norm'].std():.4f}")

## Conclusions

### Validation Passed

1. **Normalization Working**: Z-scores centered at 0 with std ≈ 1 for all systems
2. **Scale Independence**: Normalized divergence in 10-20 range regardless of raw scale (100s to millions)
3. **Chaos Detection**: All systems show positive Lyapunov exponents and Hurst ≈ 1.0
4. **Dual Topology**: Both raw and normalized fields generated correctly

### Usage

```bash
# Generate vectors with normalization
python -m prism.entry_points.signal_vector --signal --domain lorenz

# Laplace on raw (absolute energy)
python -m prism.entry_points.laplace --level signal --value-col metric_value

# Laplace on normalized (anomaly detection)
python -m prism.entry_points.laplace --level signal --value-col metric_value_norm
```